# DimABSA 2026 - DESS Model with VA Regression
## Dimensional Aspect Sentiment Triplet Extraction (Subtask 2)

**Task**: Extract (Aspect, Opinion, VA) triplets with continuous Valence-Arousal scores

**Dataset**: Combined Restaurant + Laptop (3,727 training samples)

**Model**: DESS with VA regression head (2 outputs instead of 3-class classification)

## 1. Clone Repository from GitHub
### ⚠️ IMPORTANT: Make sure you're in /kaggle/working before cloning

In [ ]:
# Ensure we're in the correct directory
%cd /kaggle/working
!pwd

# Clone repository
!git clone https://github.com/VishalRepos/dimabsa-2026.git
%cd dimabsa-2026/DESS/Codebase

# Verify location and latest commits
!echo "\n=== Current Directory ==="
!pwd
!echo "\n=== Latest Commits ==="
!git log --oneline -5

## 2. Check GPU Availability

In [ ]:
!nvidia-smi

## 3. Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch==2.1.0
!pip install -q transformers==4.36.0
!pip install -q numpy==1.26.4
!pip install -q scikit-learn==1.3.2
!pip install -q tqdm==4.65.0
!pip install -q tensorboardX==2.6
!pip install -q matplotlib==3.8.0
!pip install -q torch_geometric==2.4.0

# Restart kernel
import os
os.kill(os.getpid(), 9)

## 4. Verify Setup

In [ ]:
import torch
import numpy as np
import transformers

print("=== Package Versions ===")
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"NumPy: {np.__version__}")

print("\n=== GPU Information ===")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 5. Verify Data Files

In [ ]:
import json

# Check data directory
!ls -lh data/dimabsa_combined/

# Load and verify data
train_data = json.load(open('data/dimabsa_combined/train_dep_triple_polarity_result.json'))
test_data = json.load(open('data/dimabsa_combined/test_dep_triple_polarity_result.json'))

print(f"\n=== Data Verification ===")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# Check sample structure
sample = train_data[0]
print(f"\nSample structure:")
print(f"  Tokens: {len(sample['tokens'])}")
print(f"  Entities: {len(sample['entities'])}")
print(f"  Sentiments: {len(sample['sentiments'])}")
if sample['sentiments']:
    print(f"  VA format: {sample['sentiments'][0]['type']}")

## 6. Training Configuration

In [ ]:
# Training parameters
DATASET = "dimabsa_combined"
EPOCHS = 10
BATCH_SIZE = 4
MAX_SPAN_SIZE = 10
SEED = 42

print("=== Training Configuration ===")
print(f"Dataset: {DATASET}")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Max Span Size: {MAX_SPAN_SIZE}")
print(f"Seed: {SEED}")

## 7. Run Training - VA Regression
### ✅ All model parameters now use correct defaults (deberta-v3-base, 768 dims)
### Expected: Model learns to predict continuous VA scores [1.0-9.0]

In [ ]:
# Simplified training command - defaults are now correct
!python train.py \
    --dataset dimabsa_combined \
    --epochs 10 \
    --batch_size 4 \
    --max_span_size 10 \
    --seed 42

## 8. View Training Logs

In [ ]:
!ls -lh log/dimabsa_combined/
!echo "\n=== Latest Training Log ==="
!tail -50 log/dimabsa_combined/train_*.log

## 9. Check Best Model

In [ ]:
!ls -lh savemodels/dimabsa_combined/

## 10. Evaluate Results

In [ ]:
import json
import os

# Check if training info exists
training_info_path = 'savemodels/dimabsa_combined/final_model/training_info.json'

if os.path.exists(training_info_path):
    with open(training_info_path, 'r') as f:
        info = json.load(f)
    
    print("=== Training Summary ===")
    print(f"Dataset: {info.get('dataset', 'N/A')}")
    print(f"Model: {info.get('model', 'N/A')}")
    print(f"Epochs: {info.get('epochs', 'N/A')}")
    print(f"Batch Size: {info.get('batch_size', 'N/A')}")
    print(f"\n=== Best Performance ===")
    print(f"Best F1 Score: {info.get('best_f1', 0):.2f}%")
    
    # Check model files
    model_dir = 'savemodels/dimabsa_combined/final_model'
    print(f"\n=== Model Files ===")
    if os.path.exists(model_dir):
        files = os.listdir(model_dir)
        for f in sorted(files):
            size = os.path.getsize(os.path.join(model_dir, f)) / (1024*1024)
            print(f"  {f}: {size:.2f} MB")
    
    print(f"\n✓ Model ready at: {os.path.abspath(model_dir)}")
else:
    print("❌ Training info not found. Training may not have completed.")
    print("   Check if training finished successfully.")

## 11. Download Trained Model

In [ ]:
# List saved models
!ls -lh savemodels/dimabsa_combined/

print("\n=== Download Instructions ===")
print("1. Go to Output panel (right side)")
print("2. Navigate to savemodels/dimabsa_combined/")
print("3. Download the best model file")
print("4. Also download training logs from log/dimabsa_combined/")

## 12. Training Summary

### Model Configuration:
- **Base Model**: DeBERTa-v3-base (768 dimensions)
- **Task**: VA Regression (2 outputs: valence, arousal)
- **Loss**: MSE Loss for continuous prediction
- **Dataset**: 3,727 training samples (restaurant + laptop)
- **All defaults**: Now correctly set for deberta-v3-base

### Recent Fixes (2026-01-18):
✅ Changed all hardcoded deberta-v2-xxlarge references to deberta-v3-base
✅ Updated dimension defaults from 1536 to 768
✅ Fixed config loading to use args.pretrained_deberta_name
✅ Simplified training command (explicit parameters no longer required)

### Expected Output:
- Trained model checkpoint
- Training logs with loss curves
- Evaluation metrics

### Next Steps:
1. Download trained model
2. Run inference on test data
3. Generate submission file
4. Submit to DimABSA competition

### Files to Download:
```
savemodels/dimabsa_combined/best_model.pt
log/dimabsa_combined/train_*.log
log/dimabsa_combined/results_*.json
```

### Troubleshooting:
- If you see dimension mismatch errors, ensure you cloned the latest version
- Check git log shows commit: "Fix: Change all defaults to deberta-v3-base"
- Verify you're in /kaggle/working/dimabsa-2026/DESS/Codebase